In [1]:
import psycopg2
from psycopg2 import sql
from psycopg2 import Error

In [2]:
DB_HOST = 'pavlov.ing.puc.cl'
DB_PORT = '5432'
DB_USER = 'grupo8'
DB_PASSWORD = 'bulla8'
DB_NAME = 'grupo8e2'

In [42]:
#Muestre todos los restaurantes que ofrezcan un plato específico (string) ingresado y que esté disponible
def consulta1(): 
    #query devuelve nombre y sucursal de los restaurantes que ofrecen el plato ingresado
    query = """SELECT Restaurant.id, Restaurant.nombre, Restaurant.sucursal 
        FROM Plato
        JOIN Platorestaurant ON Plato.id = Platorestaurant.plato_id 
        JOIN Restaurant ON Platorestaurant.restaurant_id = Restaurant.id
        WHERE Plato.nombre = %(nombre)s 
        AND Plato.disponibilidad = TRUE """
    return query

#Muestre todos los pedidos de un usuario específico ingresado (email string) y su gasto mensual (solo los pedidos concretados)
def consulta2():
    query = """SELECT * 
        FROM Pedido
        JOIN Usuario ON Pedido.usuario_id = Usuario.id
        WHERE Usuario.email = %(email)s"""
    
    query1 = """ SELECT Usuario.email,  (SUM(Empresa.precio_mensual) + SUM(Empresa.precio_anual / 12) + gasto_mensual_sin_suscripcion.gasto_mensual) as gasto_mensual 
            FROM Usuario, Suscripcion, Empresa, (SELECT Usuario.email as email_usuario, (SUM(gasto_comida.precio_comida) + SUM(gasto_comida.precio_despacho)) as gasto_mensual
                                        FROM Usuario, Suscripcion, Pedido,
                                        (SELECT Pedido.id as pedido_id, Pedido.usuario_id, Empresa.id as empresa_id, Empresa.precio_despacho, SUM(Plato.precio * Plato.cantidad) as precio_comida 
                                            FROM Pedido
                                            JOIN Despacho ON Despacho.pedido_id = Pedido.id
                                            JOIN Empresa ON Despacho.empresa_id = Empresa.id
                                            JOIN ContenidoPedido ON ContenidoPedido.pedido_id = Pedido.id
                                            JOIN Plato ON ContenidoPedido.plato_id = Plato.id
                                            WHERE Pedido.estado = 'entregado a cliente' 
                                            AND Pedido.fecha BETWEEN %(fecha_inicio)s AND %(fecha_fin)s
                                            GROUP BY Pedido.id, Pedido.usuario_id, Empresa.id, Empresa.precio_despacho) as gasto_pedido_mes) 
                                        WHERE Usuario.id = gasto_pedido.usuario_id 
                                        AND Pedido.id = gasto_pedido.pedido_id 
                                        AND Suscripcion.usuario_id = Usuario.id
                                        AND Suscripcion.empresa_id NOT IN (SELECT empresa_id 
                                                                        FROM Suscripcion 
                                                                        WHERE Suscripcion.usuario_id = Usuario.id) 
                                                                        AND Suscripcion.estado = 'Vigente') 
                                        GROUP BY Usuario.email) as gasto_mensual_sin_suscripcion 
        WHERE gasto_mensual_sin_suscripcion.email_usuario = Usuario.email 
        AND Suscripcion.usuario_id = Usuario.id 
        AND Suscripcion.empresa_id = Empresa.id 
        AND Suscripcion.estado = 'Vigente' 
        AND Suscripcion.tipo = 'mensual' 
        GROUP BY Usuario.email"""

    return query, query1

#Muestre todos pedidos concretados y cancelados y el valor total de ellos
def consulta3():
    query = """SELECT Pedido.id, Pedido.estado, gasto_pedido_total.gasto_pedido
        FROM Pedido, (SELECT Pedido.id, (SUM(gasto_pedido_comida.precio_comida) + SUM(gasto_pedido_comida.precio_despacho)) as gasto_pedido
                    FROM Pedido, (SELECT Pedido.id as pedido_id, Empresa.precio_despacho, SUM(Plato.precio * Plato.cantidad) as precio_comida 
                                            FROM Pedido
                                            JOIN Despacho ON Despacho.pedido_id = Pedido.id
                                            JOIN Empresa ON Despacho.empresa_id = Empresa.id
                                            JOIN ContenidoPedido ON ContenidoPedido.pedido_id = Pedido.id
                                            JOIN Plato ON ContenidoPedido.plato_id = Plato.id
                                            GROUP BY Pedido.id, Empresa.precio_despacho) as gasto_pedido_comida) as gasto_pedido_total
        WHERE Pedido.estado IN ('entregado a cliente', 'cliente cancela', 'delivery cancela', 'restaurante cancela')"""
    return query

#Dado un estilo de plato ingresado por el usuario (string) , muestre todas los platos con ese tipo, los restaurantes que las ofrecen y las opciones de delivery.

def consulta4():
    query = """SELECT Plato.nombre, Res_em.nombre_rest, Res_em.sucursal, Res_em.nombre_emp 
        FROM Plato, (SELECT Restaurant.nombre, Restaurant.sucursal, Empresa.nombre 
                    FROM Restaurant, 
                    JOIN Restaurant_empresa ON Restaurant.id = Restaurant_empresa.restaurant_id
                    JOIN Empresa ON Empresa.id = Restaurant_empresa.empresa_id) as Res_em 
        WHERE Plato.estilo = %(estilo)s 
        AND Plato.restaurant_id = Res_em.id 
        AND Plato.disponibilidad = TRUE"""
    return query

#Dado un estilo de plato ingresado (string) por el usuario, muestre todas las platos que pertenezcan a ese estilo y sus restricciones
def consulta5():
    query = """SELECT Restaurant.nombre, Restaurant.sucursal, Plato.nombre, Plato.restriccion
        FROM Plato
        JOIN Platorestaurant ON Plato.id = Platorestaurant.plato_id 
        JOIN Restaurant ON Platorestaurant.restaurant_id = Restaurant.id
        WHERE Plato.estilo = %(estilo)s 
        AND Plato.disponibilidad = TRUE"""
    return query

#Dado un cliente ingresado por el usuario (email string), muestre todas las restaurantes a las que tiene acceso con sus suscripciones.

def consulta6():
    query = """SELECT Restaurant.nombre, Restaurant.sucursal, Empresa.nombre 
        FROM Restaurant, 
        JOIN Restaurant_empresa ON Restaurant.id = Restaurant_empresa.restaurant_id
        JOIN Empresa ON Empresa.id = Restaurant_empresa.empresa_id
        JOIN Suscripcion ON Empresa.id = Suscripcion.empresa_id
        JOIN Usuario ON Suscripcion.usuario_id = Usuario.id
        WHERE Usuario.email = %(email)s"""
    return query

# Muestre la suma de dinero gastada por cada cliente en pedidos no incluidas en planes de suscripcion

def consulta7():
    query = """SELECT Usuario.email, (SUM(gasto_comida.precio_comida) + SUM(gasto_comida.precio_despacho)) as gasto_total
        FROM Usuario, Suscripcion, Pedido 
        (SELECT Pedido.id as pedido_id, Pedido.usuario_id, Empresa.id as empresa_id, Empresa.precio_despacho, SUM(Plato.precio * Plato.cantidad) as precio_comida 
            FROM Pedido
            JOIN Despacho ON Despacho.pedido_id = Pedido.id
            JOIN Empresa ON Despacho.empresa_id = Empresa.id
            JOIN ContenidoPedido ON ContenidoPedido.pedido_id = Pedido.id
            JOIN Plato ON ContenidoPedido.plato_id = Plato.id
            WHERE Pedido.estado = 'entregado a cliente'  
            GROUP BY Pedido.id, Pedido.usuario_id, Empresa.id, Empresa.precio_despacho) as gasto_pedido
        WHERE Usuario.id = gasto_pedido.usuario_id 
        AND Pedido.id = gasto_pedido.pedido_id 
        AND Suscripcion.usuario_id = Usuario.id 
        AND Suscripcion.empresa_id NOT IN (SELECT empresa_id 
                                        FROM Suscripcion 
                                        WHERE Suscripcion.usuario_id = Usuario.id 
                                        AND Suscripcion.estado = 'Vigente') 
        GROUP BY Usuario.email"""
    return query

#Muestre todos los platos y los restaurantes que los ofrecen

def consulta8():
    query = """SELECT Plato.nombre, Restaurant.nombre, Restaurant.sucursal 
        FROM Plato
        JOIN Platorestaurant ON Plato.id = Platorestaurant.plato_id 
        JOIN Restaurant ON Platorestaurant.restaurant_id = Restaurant.id
        WHERE Plato.disponibilidad = TRUE"""
    return query

#Dado un numero (1-5) ingresado por el usuario, muestre todos las evaluaciones de Clientes, Delivery, Despachador superiores o iguales a  ́el

def consulta9(tipo: str):
    query = """SELECT Pedido.id, Pedido.evaluacion_cliente 
        FROM Pedido 
        WHERE Pedido.evaluacion_cliente >= %(numero)s"""
    
    query1 = """SELECT Pedido.id, Pedido.evaluacion_servicio as evaluacion_delivery 
        FROM Pedido 
        WHERE Pedido.evaluacion_servicio >= %(numero)s"""
    
    query2 = """SELECT Pedido.id, Pedido.evaluacion_servicio as evaluacion_despachador 
        FROM Pedido 
        WHERE Pedido.evaluacion_servicio >= %(numero)s"""

    if tipo == 'cliente':
        return query
    elif tipo == 'delivery':
        return query1
    elif tipo == 'despachador':
        return query2

#Dado una un alergeno ingresado por el usuario (string), muestre todos los platos que lo contengan en sus ingredientes (Ejemplo: maní)
def consulta10():
    query = """SELECT Restaurant.nombre, Restaurant.sucursal, Plato.nombre 
        FROM Plato
        JOIN Platorestaurant ON Plato.id = Platorestaurant.plato_id 
        JOIN Restaurant ON Platorestaurant.restaurant_id = Restaurant.id
        WHERE Plato.ingredientes LIKE %(alergeno)s 
        AND Plato.disponibilidad = TRUE """
    return query

In [43]:
def consultas(numero_consulta, **kwargs):
    conn = None
    cursor = None
    
    try:
        # Conexión con la base de datos
        conn = psycopg2.connect(
            user=DB_USER,
            password=DB_PASSWORD,
            host=DB_HOST,
            port=DB_PORT,
            dbname=DB_NAME
        )
        # Crear un cursor para ejecutar consultas
        cursor = conn.cursor()

        if numero_consulta == 1:
            try:
                query = consulta1()
                nombre_plato = kwargs.get('nombre_plato')
                cursor.execute(query, {'nombre': nombre_plato})
                rows = cursor.fetchall()
                for row in rows:
                    print(row)
            except (Exception, Error) as error:
                print("Error al ejecutar query:", error)

        if numero_consulta == 2:
            try:
                query, query1 = consulta2()
                email = kwargs.get('email')
                fecha_inicio = kwargs.get('fecha_inicio')
                fecha_fin = kwargs.get('fecha_fin')
                cursor.execute(query, {'email': email})
                rows = cursor.fetchall()
                for row in rows:
                    print(row)
                cursor.execute(query1, {'fecha_inicio': fecha_inicio, 'fecha_fin': fecha_fin})
                rows = cursor.fetchall()
                print(rows)
            except (Exception, Error) as error:
                print("Error al ejecutar query:", error)

        if numero_consulta == 3:
            try:
                query = consulta3()
                cursor.execute(query)
                rows = cursor.fetchall()
                for row in rows:
                    print(row)
            except (Exception, Error) as error:
                print("Error al ejecutar query:", error)

        if numero_consulta == 4:
            try:
                query = consulta4()
                estilo = kwargs.get('estilo')
                cursor.execute(query, {'estilo': estilo})
                rows = cursor.fetchall()
                for row in rows:
                    print(row)
            except (Exception, Error) as error:
                print("Error al ejecutar query:", error)

        if numero_consulta == 5:
            try:
                query = consulta5()
                estilo = kwargs.get('estilo')
                cursor.execute(query, {'estilo': estilo})
                rows = cursor.fetchall()
                for row in rows:
                    print(row)
            except (Exception, Error) as error:
                print("Error al ejecutar query:", error)

        if numero_consulta == 6:
            try:
                query = consulta6()
                email = kwargs.get('email')
                cursor.execute(query, {'email': email})
                rows = cursor.fetchall()
                for row in rows:
                    print(row)
            except (Exception, Error) as error:
                print("Error al ejecutar query:", error)

        if numero_consulta == 7:
            try:
                query = consulta7()
                cursor.execute(query)
                rows = cursor.fetchall()
                for row in rows:
                    print(row)
            except (Exception, Error) as error:
                print("Error al ejecutar query:", error)

        if numero_consulta == 8:
            try:
                query = consulta8()
                cursor.execute(query)
                rows = cursor.fetchall()
                for row in rows:
                    print(row)
            except (Exception, Error) as error:
                print("Error al ejecutar query:", error)
        
        if numero_consulta == 9:
            try:
                tipo = kwargs.get('tipo')
                query = consulta9(tipo)
                numero = kwargs.get('numero')
                cursor.execute(query, {'numero': numero})
                rows = cursor.fetchall()
                for row in rows:
                    print(row)
            except (Exception, Error) as error:
                print("Error al ejecutar query:", error)

        if numero_consulta == 10:
            try:
                query = consulta10()
                alergeno = kwargs.get('alergeno')
                cursor.execute(query, {'alergeno': '%{}%'.format(alergeno)}) #revisar si esto funciona
                rows = cursor.fetchall()
                for row in rows:
                    print(row)
            except (Exception, Error) as error:
                print("Error al ejecutar query:", error)

    except (Exception, Error) as error:
        print("Error al conectarse al servidor:", error)
    finally:
        if cursor:
            cursor.close()
        if conn:
            conn.close()

In [44]:
consultas(1, nombre_plato='Pizza Margarita')

(129, 'Papa Juan', 'Estado')
(130, 'Papa Juan', 'Las Condes')
(131, 'Papa Juan', 'Maintenci\x97n')
(132, 'Papa Juan', 'Grecia')
(133, 'Papa Juan', 'Estado')
(134, 'Papa Juan', 'Apoquindo')
(135, 'Papa Juan', 'Viel')
(136, 'Papa Juan', 'OHiggins')
(137, 'Papa Juan', 'Prat')
(162, 'Just Pizza', 'Maintenci\x97n')
(163, 'Just Pizza', 'Grecia')
(164, 'Just Pizza', 'Estado')
(165, 'Just Pizza', 'Apoquindo')
(166, 'Just Pizza', 'Viel')
(167, 'Just Pizza', 'OHiggins')
(168, 'Just Pizza', 'Prat')
(169, 'Just Pizza', 'Alemania')
(170, 'Just Pizza', 'Larga')
(171, 'Just Pizza', 'Col\x97n')
(85, 'Doble Queso', 'Estado')
(86, 'Doble Queso', 'Apoquindo')
(87, 'Doble Queso', 'Viel')
(88, 'Doble Queso', 'OHiggins')
(89, 'Doble Queso', 'Prat')
(90, 'Doble Queso', 'Alemania')
(85, 'Doble Queso', 'Estado')
(86, 'Doble Queso', 'Apoquindo')
(87, 'Doble Queso', 'Viel')
(88, 'Doble Queso', 'OHiggins')
(89, 'Doble Queso', 'Prat')
(90, 'Doble Queso', 'Alemania')
(150, 'New Sushi', 'Apoquindo')
(151, 'New Sushi

In [45]:
consultas(5, estilo='Internacional')

('Casiss', 'OHiggins', 'Enchiladas', 'diabeticos')
('Casiss', 'Viel', 'Enchiladas', 'diabeticos')
('Casiss', 'Apoquindo', 'Enchiladas', 'diabeticos')
('Casiss', 'Estado', 'Enchiladas', 'diabeticos')
('Casiss', 'Grecia', 'Enchiladas', 'diabeticos')
('Casiss', 'Maintenci\x97n', 'Enchiladas', 'diabeticos')
('Casiss', 'Las Condes', 'Enchiladas', 'diabeticos')
('DeliQuick', 'San Antonio', 'Spaghetti Carbonara', 'alergicos')
('DeliQuick', 'Apoquindo', 'Spaghetti Carbonara', 'alergicos')
('DeliQuick', 'Vitacura', 'Spaghetti Carbonara', 'alergicos')
('DeliQuick', 'Col\x97n', 'Spaghetti Carbonara', 'alergicos')
('Chick-Fil-A', 'Alemania', 'Tacos al Pastor', 'celiacos')
('Chick-Fil-A', 'Prat', 'Tacos al Pastor', 'celiacos')
('Chick-Fil-A', 'OHiggins', 'Tacos al Pastor', 'celiacos')
('Chick-Fil-A', 'Viel', 'Tacos al Pastor', 'celiacos')
('Chick-Fil-A', 'Grecia', 'Tacos al Pastor', 'celiacos')
('Chick-Fil-A', 'Estado', 'Tacos al Pastor', 'celiacos')
('Chick-Fil-A', 'Apoquindo', 'Tacos al Pastor', 

In [46]:
consultas(8)

('DCCHamburguer', 'DCCRestaurant', 'Las Parcelas')
('Pizza Margarita', 'Papa Juan', 'Prat')
('Pizza Margarita', 'Papa Juan', 'OHiggins')
('Pizza Margarita', 'Papa Juan', 'Viel')
('Pizza Margarita', 'Papa Juan', 'Apoquindo')
('Pizza Margarita', 'Papa Juan', 'Estado')
('Pizza Margarita', 'Papa Juan', 'Grecia')
('Pizza Margarita', 'Papa Juan', 'Maintenci\x97n')
('Pizza Margarita', 'Papa Juan', 'Las Condes')
('Pizza Margarita', 'Papa Juan', 'Estado')
('Pizza Margarita', 'Just Pizza', 'Col\x97n')
('Pizza Margarita', 'Just Pizza', 'Larga')
('Pizza Margarita', 'Just Pizza', 'Alemania')
('Pizza Margarita', 'Just Pizza', 'Prat')
('Pizza Margarita', 'Just Pizza', 'OHiggins')
('Pizza Margarita', 'Just Pizza', 'Viel')
('Pizza Margarita', 'Just Pizza', 'Apoquindo')
('Pizza Margarita', 'Just Pizza', 'Estado')
('Pizza Margarita', 'Just Pizza', 'Grecia')
('Pizza Margarita', 'Just Pizza', 'Maintenci\x97n')
('Pizza Margarita', 'Doble Queso', 'Alemania')
('Pizza Margarita', 'Doble Queso', 'Prat')
('Pizza

In [40]:
consultas(10, alergeno = 'tomate')

('DCCRestaurant', 'Las Parcelas', 'DCCHamburguer')
('Papa Juan', 'Prat', 'Pizza Margarita')
('Papa Juan', 'OHiggins', 'Pizza Margarita')
('Papa Juan', 'Viel', 'Pizza Margarita')
('Papa Juan', 'Apoquindo', 'Pizza Margarita')
('Papa Juan', 'Estado', 'Pizza Margarita')
('Papa Juan', 'Grecia', 'Pizza Margarita')
('Papa Juan', 'Maintenci\x97n', 'Pizza Margarita')
('Papa Juan', 'Las Condes', 'Pizza Margarita')
('Papa Juan', 'Estado', 'Pizza Margarita')
('Just Pizza', 'Col\x97n', 'Pizza Margarita')
('Just Pizza', 'Larga', 'Pizza Margarita')
('Just Pizza', 'Alemania', 'Pizza Margarita')
('Just Pizza', 'Prat', 'Pizza Margarita')
('Just Pizza', 'OHiggins', 'Pizza Margarita')
('Just Pizza', 'Viel', 'Pizza Margarita')
('Just Pizza', 'Apoquindo', 'Pizza Margarita')
('Just Pizza', 'Estado', 'Pizza Margarita')
('Just Pizza', 'Grecia', 'Pizza Margarita')
('Just Pizza', 'Maintenci\x97n', 'Pizza Margarita')
('Doble Queso', 'Alemania', 'Pizza Margarita')
('Doble Queso', 'Prat', 'Pizza Margarita')
('Doble